# GloVe Word Embeddings Demo

This demo was part of a presentation for [this word embeddings workshop](https://www.eventbrite.com/e/practical-ai-for-female-engineers-product-managers-and-designers-tickets-34805104003) and a talk at [the Demystifying AI conference](https://www.eventbrite.com/e/demystifying-deep-learning-ai-tickets-34351888423).  It is not necessary to download the demo to be able to follow along and enjoy the workshop.

It is available on Github at https://github.com/fastai/word-embeddings-workshop

## Loading our data

### Imports

In [1]:
import pickle
import numpy as np
import re
import json

In [2]:
np.set_printoptions(precision=4, suppress=True)

The dataset is available at http://files.fast.ai/models/glove/6B.100d.tgz
To download and unzip the files from the command line, you can run:

    wget http://files.fast.ai/models/glove_50_glove_100.tgz 
    tar xvzf glove_50_glove_100.tgz

You will need to update the path below to be accurate for where you are storing the data.

In [3]:
vecs = np.load("/home/scasanova/Downloads/glove_50_glove_100/glove_vectors_100d.npy")
vecs50 = np.load("/home/scasanova/Downloads/glove_50_glove_100/glove_vectors_50d.npy")

In [4]:
with open('/home/scasanova/Downloads/glove_50_glove_100/words.txt') as f:
    content = f.readlines()
words = [x.strip() for x in content] 

In [5]:
wordidx = json.load(open('/home/scasanova/Downloads/glove_50_glove_100/wordsidx.txt'))

### What the data looks like

Let's see what our data looks like:

In [6]:
len(words)

400000

In [7]:
words[:10]

['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s"]

In [8]:
words[600:610]

['together',
 'congress',
 'index',
 'australia',
 'results',
 'hard',
 'hours',
 'land',
 'action',
 'higher']

wordidx allows us to look up a word in order to find out it's index:

In [9]:
type(wordidx)

dict

In [10]:
wordidx['feminist']

11853

In [11]:
words[11853]

'feminist'

## Words as vectors

The word "intelligence" is represented by the 100 dimensional vector:

In [12]:
type(vecs)

numpy.ndarray

In [13]:
vecs[11853]

array([ 0.296 ,  0.7626, -0.9866,  0.3776,  0.3194,  0.8286, -0.1686,
       -1.4558,  0.1965,  0.3854, -0.3348, -0.6503, -0.2528, -0.11  ,
       -0.1545,  0.5354, -0.4527, -0.0516,  0.1312,  0.0744,  0.5001,
        0.2151,  0.0688,  0.4347,  0.261 , -0.0371,  0.1385, -1.518 ,
        0.0641,  0.149 , -0.0314,  0.5038,  0.2839,  0.3457, -0.4411,
       -0.3459, -0.2118,  0.5651, -0.088 , -0.0438, -1.2228,  0.6039,
       -0.23  ,  0.2287, -0.2695, -0.9398,  0.2376,  0.3302, -0.2422,
        0.6359,  0.1347,  0.5542,  0.1432,  0.2861,  0.0216, -0.7437,
        0.3508,  0.362 ,  0.5566,  0.3403,  0.3613,  0.5185, -0.5437,
       -0.285 ,  1.1831, -0.1192,  0.2473,  0.0614,  0.4436, -0.244 ,
        0.2016,  0.5143, -0.4695, -0.0974, -0.9836, -0.3594,  0.3903,
       -0.517 , -0.1659, -1.2132, -1.3228,  0.0578,  0.7022,  0.3492,
       -0.9103, -0.381 , -0.1545,  0.4467, -0.009 , -0.9838,  1.0114,
       -0.227 ,  0.2697,  0.1566,  0.5613,  0.1175, -0.5755, -0.6324,
        0.1052,  1.2

This lets us do some useful calculations. For instance, we can see how far apart two words are using a distance metric:

In [14]:
from scipy.spatial.distance import cosine as dist

Smaller numbers mean two words are closer together, larger numbers mean they are further apart.

The distance between similar words is low:

In [15]:
dist(vecs[wordidx["puppy"]], vecs[wordidx["dog"]])

0.27636247873306274

In [16]:
dist(vecs[wordidx["queen"]], vecs[wordidx["princess"]])

0.20527541637420654

And the distance between unrelated words is high:

In [17]:
dist(vecs[wordidx["celebrity"]], vecs[wordidx["dusty"]])

0.9883578838780522

In [18]:
dist(vecs[wordidx["kitten"]], vecs[wordidx["airplane"]])

0.8729851841926575

In [19]:
dist(vecs[wordidx["avalanche"]], vecs[wordidx["antique"]])

0.9621107056736946

### Bias

There is a lot of opportunity for bias:

In [20]:
dist(vecs[wordidx["man"]], vecs[wordidx["genius"]])

0.5098515152931213

In [21]:
dist(vecs[wordidx["woman"]], vecs[wordidx["genius"]])

0.689783364534378

Not all pairs are stereotyped:

In [22]:
dist(vecs[wordidx["man"]], vecs[wordidx["emotional"]])

0.5595748424530029

In [23]:
dist(vecs[wordidx["woman"]], vecs[wordidx["emotional"]])

0.6257205307483673

I just checked the distance between pairs of words, because this is a quick and simple way to illustrate the concept.  It is also a very **noisy** approach, and **researchers approach this problem in more systematic ways**.

## Visualizing the words

We will use [Plotly](https://plot.ly/), a Python library to make interactive graphs (note: everything below is done with the free, offline version of Plotly).

### Methods

In [24]:
import plotly
import plotly.graph_objs as go    
from IPython.display import IFrame

In [25]:
def plotly_3d(Y, cat_labels):
    trace_dict = {}
    for i, label in enumerate(cat_labels):
        trace_dict[i] = go.Scatter3d(
            x=Y[i*5:(i+1)*5, 0],
            y=Y[i*5:(i+1)*5, 1],
            z=Y[i*5:(i+1)*5, 2],
            mode='markers',
            marker=dict(
                size=8,
                line=dict(
                    color='rgba('+ str(i*40) + ',' + str(i*40) + ',' + str(i*40) + ', 0.14)',
                    width=0.5
                ),
                opacity=0.8
            ),
            text = my_words[i*5:(i+1)*5],
            name = label
        )

    data = [item for item in trace_dict.values()]
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )

    plotly.offline.plot({
        "data": data,
        "layout": layout
    })

In [26]:
def plotly_2d(Y, cat_labels):
    trace_dict = {}
    for i, label in enumerate(cat_labels):
        trace_dict[i] = go.Scatter(
            x=Y[i*5:(i+1)*5, 0],
            y=Y[i*5:(i+1)*5, 1],
            mode='markers',
            marker=dict(
                size=8,
                line=dict(
                    color='rgba('+ str(i*40) + ',' + str(i*40) + ',' + str(i*40) + ', 0.14)',
                    width=0.5
                ),
                opacity=0.8
            ),
            text = my_words[i*5:(i+1)*5],
            name = label
        )

    data = [item for item in trace_dict.values()]
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )

    plotly.offline.plot({
        "data": data,
        "layout": layout
    })

### Preparing the Data

Let's plot words from a few different categories:

In [27]:
categories = [
              "bugs", "music", 
              "pleasant", "unpleasant", 
              "science", "arts"
             ]

In [28]:
my_words = [
            "maggot", "flea", "tarantula", "bedbug", "mosquito", 
            "violin", "cello", "flute", "harp", "mandolin",
            "joy", "love", "peace", "pleasure", "wonderful",
            "agony", "terrible", "horrible", "nasty", "failure", 
            "physics", "chemistry", "science", "technology", "engineering",
            "poetry", "art", "literature", "dance", "symphony",
           ]

Again, we need to look up the indices of our words using the wordidx dictionary:

In [29]:
X = np.array([wordidx[word] for word in my_words])

In [30]:
vecs[X].shape

(30, 100)

Now, we will make a set combining our words with the first 10,000 words in our entire set of words (some of the words will already be in there), and create a matrix of their embeddings.

In [31]:
embeddings = np.concatenate((vecs[X], vecs[:10000,:]), axis=0); embeddings.shape

(10030, 100)

### Viewing the words in 3D

The words are in 100 dimensions, so we will need a way to reduce them to 3 dimensions so that we can view them.  Two good options are T-SNE or PCA.  The main idea is to find a meaningful way to go from 100 dimensions to 3 dimensions (while keeping a similar notion of what is close to what).

You would typically just use one of these (T-SNE or PCA).  I've included both if you're interested.

#### TSNE

In [32]:
from sklearn import manifold

In [ ]:
tsne = manifold.TSNE(n_components=3, init='pca', random_state=0)
Y = tsne.fit_transform(embeddings)
plotly_3d(Y, categories)

In [ ]:
IFrame('temp-plot.html', width=600, height=400)

#### PCA

In [ ]:
from sklearn import decomposition

In [ ]:
pca = decomposition.PCA(n_components=3).fit(subset.T)
components = pca.components_
plotly_3d(components.T[:len(my_words),:], categories)

In [ ]:
IFrame('temp-plot.html', width=600, height=400)

## Nearest Neighbors

We can also see what words are close to a given word.

In [ ]:
from sklearn.neighbors import NearestNeighbors

Nearest Neighbors is an algorithm that finds the points closest to a given point.

In [ ]:
neigh = NearestNeighbors(n_neighbors=10, radius=0.5, metric='cosine', algorithm='brute')
neigh.fit(vecs) 

In [ ]:
distances, indices = neigh.kneighbors([vecs[wordidx["feminist"]]])

In [ ]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

We can take this a step further, and add two words together.  What is the result?

In [ ]:
new_vec = vecs[wordidx["artificial"]] + vecs[wordidx["intelligence"]]

In [ ]:
new_vec

In [ ]:
distances, indices = neigh.kneighbors([new_vec])

In [ ]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

In [ ]:
distances, indices = neigh.kneighbors([vecs[wordidx["king"]]])

In [ ]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

In [ ]:
new_vec = vecs[wordidx["king"]] - vecs[wordidx["he"]] + vecs[wordidx["she"]]

In [ ]:
distances, indices = neigh.kneighbors([new_vec])

In [ ]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

In [ ]:
wordidx["programmer"]

In [ ]:
distances, indices = neigh.kneighbors([vecs[wordidx["programmer"]]])

Closest words to "programmer":

In [ ]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

Feminine version of "programmer"

In [ ]:
new_vec = vecs[wordidx["programmer"]] - vecs[wordidx["he"]] + vecs[wordidx["she"]]

In [ ]:
distances, indices = neigh.kneighbors([new_vec])

In [ ]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

Masculine version of "programmer"

In [ ]:
new_vec = vecs[wordidx["programmer"]] - vecs[wordidx["she"]] + vecs[wordidx["he"]]

In [ ]:
distances, indices = neigh.kneighbors([new_vec])

In [ ]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

In [ ]:
distances, indices = neigh.kneighbors([vecs[wordidx["doctor"]]])

In [ ]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

Feminine version of doctor:

In [ ]:
new_vec = vecs[wordidx["doctor"]] - vecs[wordidx["he"]] + vecs[wordidx["she"]]

In [ ]:
distances, indices = neigh.kneighbors([new_vec])

In [ ]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

Masculine version of doctor:

In [ ]:
new_vec = vecs[wordidx["doctor"]] - vecs[wordidx["she"]] + vecs[wordidx["he"]]

In [ ]:
distances, indices = neigh.kneighbors([new_vec])

In [ ]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

## Bias

Again, just looking at individual words is a **noisy** approach (I'm using it as a simple illustration).  [Researchers from Princeton and University of Bath](https://www.princeton.edu/~aylinc/papers/caliskan-islam_semantics.pdf) use **small baskets of terms** to represent concepts.  They first confirmed that flowers are more pleasant than insects, and musical instruments are more pleasant from weapons.

They then found that European American names are "more pleasant" than African American names, as captured by how close the word vectors are (as embedded by GloVe, which is a library from Stanford, along the same lines as Word2Vec).

    We show for the first time that if AI is to exploit via our language the vast 
    knowledge that culture has compiled, it will inevitably inherit human-like 
    prejudices. In other words, if AI learns enough about the properties of language 
    to be able to understand and produce it, it also acquires cultural associations 
    that can be offensive, objectionable, or harmful.

[Researchers from Boston University and Microsoft Research](https://arxiv.org/pdf/1606.06121.pdf) found the pairs most analogous to *He : She*.  They found gender bias, and also proposed a way to debias the vectors.

Rob Speer, CTO of Luminoso, tested for ethnic bias by finding correlations for a list of positive and negative words:

    The tests I implemented for ethnic bias are to take a list of words, such as 
    “white”, “black”, “Asian”, and “Hispanic”, and find which one has the strongest 
    correlation with each of a list of positive and negative words, such as “cheap”, 
    “criminal”, “elegant”, and “genius”. I did this again with a fine-grained version 
    that lists hundreds of words for ethnicities and nationalities, and thus is more 
    difficult to get a low score on, and again with what may be the trickiest test of 
    all, comparing words for different religions and spiritual beliefs.

**Ways to address bias**

There are a few different approaches:

- Debias word embeddings
  - [Technique in Bolukbasi, et al.](https://arxiv.org/abs/1606.06121)
  - [ConceptNet Numberbatch (Rob Speer)](https://blog.conceptnet.io/2017/04/24/conceptnet-numberbatch-17-04-better-less-stereotyped-word-vectors/)
- Argument that “awareness is better than blindness”: debiasing should happen at time of action, not at perception. ([Caliskan-Islam, Bryson, Narayanan](https://www.princeton.edu/~aylinc/papers/caliskan-islam_semantics.pdf))

Either way, you need to be on the lookout for bias and have a plan to address it!

If you are interested in the topic of bias in AI, I gave a workshop [you can watch here](https://www.youtube.com/watch?v=25nC0n9ERq4) that covers this material and goes into more depth about bias.

# Movie Reviews Sentiment Analysis Demo

This demo has been adapted (and simplified) from part of Lesson 5 of [Practical Deep Learning for Coders](http://course.fast.ai/index.html)

## Setup data

We're going to look at the IMDB dataset, which contains movie reviews from IMDB, along with their sentiment. 

We will be using [Keras](https://keras.io/), a high-level neural network API. Two of the guiding principles of Keras are **user-friendliness** (it's designed for humans, not machines) and **works with Python**.  Yay for both of these!

Keras can run on top of many other neural network frameworks, including TensorFlow, Theano, R, MxNet, or CNTK.  I am using it on top of TensorFlow here.

Keras comes with some helpers for the IMDB dataset.

In [ ]:
from keras.datasets import imdb
from keras.utils.data_utils import get_file
idx = imdb.get_word_index()

In [ ]:
import keras.backend as K

def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    cfg.gpu_options.per_process_gpu_memory_fraction = 0.6
    K.set_session(K.tf.Session(config=cfg))
    
limit_mem()

This is the word list:

In [ ]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

...and this is the mapping from id to word

In [ ]:
idx2word = {v: k for k, v in idx.items()}

We download the reviews using code from https://keras.io/datasets/:

In [ ]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [ ]:
len(x_train)

Here's the 1st review. As you see, the words have been replaced by ids. The ids can be looked up in idx2word.

In [ ]:
', '.join(map(str, x_train[0]))

The first word of the first review is 23022. Let's see what that is.

In [ ]:
idx2word[23022]

Here's the whole review, mapped from ids to words.

In [ ]:
' '.join([idx2word[o] for o in x_train[0]])

The labels are 1 for positive, 0 for negative.

In [ ]:
labels_train[:10]

Reduce vocab size by setting rare words to max index.

In [ ]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

Look at distribution of lengths of sentences.

In [ ]:
trn[:10]

In [ ]:
lens = np.array([len(review) for review in trn])

In [ ]:
(lens.max(), lens.min(), lens.mean())

Pad (with zero) or truncate each sentence to make consistent length.

In [ ]:
from keras.preprocessing import sequence

In [ ]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

This results in nice rectangular matrices that can be passed to ML algorithms. Reviews shorter than 500 words are pre-padded with zeros, those greater are truncated.

In [ ]:
trn.shape

## Create a model

### Single conv layer with max pooling

*Convolutional neural networks* (abbreviated CNNs) are a powerful type of neural networks that do well with ordered data.  They have traditionally been used primarily for image data, but more recently are showing great results on natural language data.  [Facebook AI recently announced results](https://code.facebook.com/posts/1978007565818999/a-novel-approach-to-neural-machine-translation/) of using a CNN to speed up language translation 9x faster than the current state-of-the-art. 

We'll use a 1D CNN, since a sequence of words is 1D.

For this workshop, we will treat the CNN as a black box.  If you want to learn more about what is going on inside it, check out [Practical Deep Learning for Coders](http://course.fast.ai/) (the only pre-req is 1 year of coding experience).

In [ ]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.optimizers import Adam

In [ ]:
conv1 = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.4),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')])

In [ ]:
conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=4, batch_size=64)

In deep learning, often as you get closer to the answer, you need to reduce your *learning rate*, which is the step size for how the algorithm changes it's guess each time.  When you are far from the answer, you want to take large steps to get to the right vicinity.  Once you are close to the answer, you want to take small steps so you don't overshoot the answer.

In [ ]:
conv1.optimizer.lr=1e-4

In [ ]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=4, batch_size=64)

The [Stanford paper](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf)(2011) that this dataset is from cites a state of the art accuracy (without unlabelled data) of 88.3%.  We have surpassed that!

Note that accuracy of 88.9% means an error rate of 11.1% (it's often more helpful to talk about error rates).

### Using our GloVe word embeddings

We could improve our model by using the GloVe word embeddings from above, since this capture semantic meaning, and have been trained for much longer on a much larger dataset than what we are using here.

We are going to use a version of GloVe where the embeddings have just 50 dimensions (as opposed to 100).  It's the same idea as before.

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [ ]:
def create_emb():
    n_fact = vecs50.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs50[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = np.random.normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = np.random.normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [ ]:
emb = create_emb()

We pass our embedding matrix to the Embedding constructor, and set it to non-trainable.

In [ ]:
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.4, weights=[emb]),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=4, batch_size=64)

We decrease the learning rate now that we are getting closer to the answer.

In [ ]:
model.optimizer.lr=1e-4

In [ ]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=4, batch_size=64)

Our error rate has improved from 11.1% to 10.3%, a 7% improvement 

(this value was fluctuating, but I typically got that it was between 4-10%)

In [ ]:
(11.1 - 10.3)/11.1